In [19]:
import localsolver
import sys
import math

In [20]:
def read_elem(filename):
    with open(filename) as f:
        return [str(elem) for elem in f.read().split()]

In [21]:
def read_input_sdvrp(filename):
    file_it = iter(read_elem(filename))

    nb_customers = int(next(file_it))
    capacity = int(next(file_it))

    demands = [None] * nb_customers
    for i in range(nb_customers):
        demands[i] = int(next(file_it))

    # Extracting the coordinates of the depot and the customers
    customers_x = [None] * nb_customers
    customers_y = [None] * nb_customers
    depot_x = float(next(file_it))
    depot_y = float(next(file_it))
    for i in range(nb_customers):
        customers_x[i] = float(next(file_it))
        customers_y[i] = float(next(file_it))

    distance_matrix = compute_distance_matrix(customers_x, customers_y)
    distance_depots = compute_distance_depots(depot_x, depot_y, customers_x, customers_y)
    return nb_customers, capacity, distance_matrix, distance_depots, demands

In [22]:
data = read_input_sdvrp(r'./instances/S51D4.sd')

In [23]:
def main(instance_file, str_time_limit, sol_file):
    #
    # Read instance data
    #
    nb_customers, truck_capacity, dist_matrix_data, dist_depot_data, demands_data = \
        read_input_sdvrp(instance_file)
    nb_trucks = nb_customers

    with localsolver.LocalSolver() as ls:
        #
        # Declare the optimization model
        #
        model = ls.model

        # Sequence of customers visited by each truck
        customers_sequences = [model.list(nb_customers) for _ in range(nb_trucks)]

        # Quantity carried by each truck for each customer
        quantity = [None] * nb_trucks
        for k in range(nb_trucks):
            quantity[k] = [model.float(0, demands_data[i]) for i in range(nb_customers)]

        # All customers must be visited at least by one truck
        model.constraint(model.cover(customers_sequences))

        # Create LocalSolver arrays to be able to access them with "at" operators
        dist_matrix = model.array(dist_matrix_data)
        dist_depot = model.array(dist_depot_data)

        route_distances = [None] * nb_trucks
        trucks_used = [None] * nb_trucks

        for k in range(nb_trucks):
            sequence = customers_sequences[k]
            c = model.count(sequence)

            # A truck is used if it visits at least one customer
            trucks_used[k] = model.count(sequence) > 0

            # The quantity carried in each route must not exceed the truck capacity
            quantity_array = model.array(quantity[k])
            quantity_lambda = model.lambda_function(lambda j: quantity_array[j])
            route_quantity = model.sum(sequence, quantity_lambda)
            model.constraint(route_quantity <= truck_capacity)

            # Distance traveled by each truck
            dist_lambda = model.lambda_function(
                lambda i: model.at(dist_matrix, sequence[i - 1], sequence[i]))
            route_distances[k] = model.sum(model.range(1, c), dist_lambda) \
                + model.iif(
                    trucks_used[k],
                    dist_depot[sequence[0]] + dist_depot[sequence[c - 1]],
                    0)

        for i in range(nb_customers):
            # Each customer must receive at least its demand
            quantity_served = model.sum(
                quantity[k][i] * model.contains(customers_sequences[k], i)
                for k in range(nb_trucks))
            model.constraint(quantity_served >= demands_data[i])

        total_distance = model.sum(route_distances)

        # Objective: minimize the distance traveled
        model.minimize(total_distance)

        model.close()

        # Parameterize the solver
        ls.param.time_limit = int(str_time_limit)

        ls.solve()

        #
        # Write the solution in a file with the following format:
        # each line k contain the customers visited by the truck k
        #
        with open(sol_file, 'w') as f:
            f.write("%d \n" % total_distance.value)
            for k in range(nb_trucks):
                if trucks_used[k].value != 1:
                    continue
                # Values in sequence are in 0...nbCustomers. +1 is to put it back in 1...nbCustomers+1
                for customer in customers_sequences[k].value:
                    f.write("%d " % (customer + 1))
                f.write("\n")

In [24]:





def read_input_sdvrp(filename):
    file_it = iter(read_elem(filename))

    nb_customers = int(next(file_it))
    capacity = int(next(file_it))

    demands = [None] * nb_customers
    for i in range(nb_customers):
        demands[i] = int(next(file_it))

    # Extracting the coordinates of the depot and the customers
    customers_x = [None] * nb_customers
    customers_y = [None] * nb_customers
    depot_x = float(next(file_it))
    depot_y = float(next(file_it))
    for i in range(nb_customers):
        customers_x[i] = float(next(file_it))
        customers_y[i] = float(next(file_it))

    distance_matrix = compute_distance_matrix(customers_x, customers_y)
    distance_depots = compute_distance_depots(depot_x, depot_y, customers_x, customers_y)
    return nb_customers, capacity, distance_matrix, distance_depots, demands


# Compute the distance between two customers
def compute_distance_matrix(customers_x, customers_y):
    nb_customers = len(customers_x)
    distance_matrix = [[None for _ in range(nb_customers)] for _ in range(nb_customers)]
    for i in range(nb_customers):
        distance_matrix[i][i] = 0
        for j in range(nb_customers):
            dist = compute_dist(customers_x[i], customers_x[j], customers_y[i], customers_y[j])
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist
    return distance_matrix


# Compute the distances to depot
def compute_distance_depots(depot_x, depot_y, customers_x, customers_y):
    nb_customers = len(customers_x)
    distance_depots = [None] * nb_customers
    for i in range(nb_customers):
        dist = compute_dist(depot_x, customers_x[i], depot_y, customers_y[i])
        distance_depots[i] = dist
    return distance_depots


def compute_dist(xi, xj, yi, yj):
    exact_dist = math.sqrt(math.pow(xi - xj, 2) + math.pow(yi - yj, 2))
    return int(math.floor(exact_dist + 0.5))

instance_file = r'./instances/S51D4.sd'
sol_file = r'./result.csv'
str_time_limit = "20"

main(instance_file, str_time_limit, sol_file)


Push initial solution 100%
Model:  expressions = 9055, decisions = 2550, constraints = 101, objectives = 1
Param:  time limit = 20 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 51)
[  1 sec,   40000 itr]:         1629
[  2 sec,   69421 itr]:         1628
[  3 sec,   97034 itr]:         1622
[  4 sec,   97034 itr]:         1622
[  5 sec,  160000 itr]:         1618
[  6 sec,  191837 itr]:         1614
[  7 sec,  219138 itr]:         1613
[  8 sec,  245337 itr]:         1613
[  9 sec,  245337 itr]:         1613
[ 10 sec,  314093 itr]:         1597
[ optimality gap     ]:      100.00%
[ 11 sec,  339788 itr]:         1596
[ 12 sec,  339788 itr]:         1596
[ 13 sec,  400000 itr]:         1596
[ 14 sec,  400000 itr]:         1596
[ 15 sec,  464395 itr]:         1588
[ 16 sec,  464395 itr]:         1588
[ 17 sec,  520000 itr]:         1584
[ 18 sec,  556478 itr]:         1583
[ 19 sec,  556478 itr]:         1583
[

In [25]:
data

(50,
 160,
 [[0,
   12,
   19,
   31,
   22,
   17,
   23,
   12,
   24,
   34,
   12,
   21,
   42,
   27,
   36,
   19,
   31,
   28,
   46,
   21,
   27,
   7,
   22,
   29,
   33,
   19,
   8,
   16,
   21,
   33,
   17,
   6,
   43,
   31,
   27,
   31,
   30,
   19,
   43,
   56,
   44,
   45,
   34,
   38,
   42,
   14,
   23,
   12,
   26,
   24],
  [12,
   0,
   15,
   37,
   21,
   28,
   35,
   22,
   16,
   28,
   11,
   25,
   50,
   38,
   35,
   9,
   34,
   36,
   51,
   12,
   15,
   11,
   34,
   41,
   43,
   29,
   19,
   19,
   9,
   24,
   23,
   11,
   39,
   20,
   19,
   24,
   32,
   15,
   35,
   62,
   50,
   48,
   46,
   39,
   40,
   20,
   29,
   25,
   21,
   14],
  [19,
   15,
   0,
   50,
   36,
   35,
   35,
   21,
   31,
   43,
   25,
   38,
   61,
   46,
   51,
   23,
   48,
   47,
   64,
   8,
   24,
   12,
   37,
   46,
   52,
   25,
   27,
   9,
   17,
   37,
   16,
   23,
   54,
   32,
   10,
   12,
   47,
   30,
   49,
   75,
   63,
   62,
   